In [1]:
import pysal as ps
import numpy as np
import scipy as sp
from gwr.sel_bw import Sel_BW
from gwr.gwr import GWR, MGWR
import pandas as pd
import pysal
import libpysal as ps
import pickle as pk
import statsmodels.formula.api as smf
import statsmodels as sm
import statsmodels.genmod.families as fam

In [2]:
data = pysal.open('/Users/toshan/Dropbox/dev/GWR4/georgia/GData_utm.csv')
coords = list(zip(data.by_col('X'), data.by_col('Y')))
y = np.array(data.by_col('PctBach')).reshape((-1,1))
rural  = np.array(data.by_col('PctRural')).reshape((-1,1))
pov = np.array(data.by_col('PctPov')).reshape((-1,1)) 
black = np.array(data.by_col('PctBlack')).reshape((-1,1))
fb = np.array(data.by_col('PctFB')).reshape((-1,1))
pop = np.array(data.by_col('TotPop90')).reshape((-1,1))


X = np.hstack([fb, black, rural])


In [3]:
X = (X - X.mean(axis=0)) / X.std(axis=0)

y = y.reshape((-1,1))

y = (y - y.mean(axis=0)) / y.std(axis=0)

sel = Sel_BW(coords, y, X, sigma2='ml')

bw = sel.search()
print(bw)
gwr = GWR(coords, y, X, bw, sigma2='v1')
gwr_results = gwr.fit()

117.0


In [4]:
X = (X - X.mean(axis=0)) / X.std(axis=0)

y = y.reshape((-1,1))

y = (y - y.mean(axis=0)) / y.std(axis=0)

selector = Sel_BW(coords, y, X, multi=True, constant=True, sigma2='ml')
bw = selector.search(bw_min=2, bw_max=159)
print(bw)
mgwr = MGWR(coords, y, X, selector, constant=True, sigma2='v1')
mgwr_results = mgwr.fit()

[92.0, 101.0, 136.0, 158.0]


In [38]:
print(gwr_results.aicc, mgwr_results.aicc)

311.31676819 311.539771245


In [20]:
print(gwr_results.tr_S, mgwr_results.tr_S)

6.32353110403 6.30916183204


In [5]:
pk.dump(mgwr_results, open('mgwr_example.p', 'wb'), protocol=2)
#This is for producing results for writing tests

In [11]:
print(gwr_results.ENP, mgwr_results.ENP)
print(mgwr_results.ENP_j)

11.8047697167 11.3682508727
[3.8446710802641428, 3.5137708051516521, 2.2580525278898254, 1.7517564593926893]


In [9]:
gwr_results.adj_alpha[1]

0.016942304237968626

In [13]:
mgwr_results.adj_alpha[1]

0.017592855949398047

In [14]:
mgwr_results.adj_alpha_j[:,1]

array([ 0.01300501,  0.01422973,  0.02214297,  0.02854278])

In [17]:
print(gwr_results.critical_tval(0.016942304237968626), gwr_results.critical_tval())

2.41353953494 2.41353953494


In [19]:
print(mgwr_results.critical_tval([ 0.01300501,  0.01422973,  0.02214297,  0.02854278]))

[ 2.5121076   2.47888785  2.31069121  2.21000184]


In [21]:
print(mgwr_results.critical_tval())

[ 2.51210749  2.47888792  2.31069113  2.21000184]


In [21]:
gwr_results.filter_tvals(critical_t=2.4135395349408113)[0:5]

array([[-3.09018967,  0.        ,  0.        , -5.27572914],
       [-3.76494511,  0.        ,  0.        , -5.39424347],
       [-3.34980705,  0.        ,  0.        , -5.39118806],
       [-2.92291757,  0.        ,  0.        , -4.32039516],
       [ 2.77789997,  8.46401973,  0.        , -2.73604891]])

In [22]:
gwr_results.filter_tvals(alpha=0.016942304237968626)[0:5]

array([[-3.09018967,  0.        ,  0.        , -5.27572914],
       [-3.76494511,  0.        ,  0.        , -5.39424347],
       [-3.34980705,  0.        ,  0.        , -5.39118806],
       [-2.92291757,  0.        ,  0.        , -4.32039516],
       [ 2.77789997,  8.46401973,  0.        , -2.73604891]])

In [23]:
mgwr_results.filter_tvals()[0:5]

array([[-2.5733093 ,  2.69323024,  0.        , -5.37412032],
       [-3.22998159,  0.        ,  0.        , -5.27052241],
       [-2.85224022,  2.53018155,  0.        , -5.36015537],
       [ 0.        ,  0.        ,  0.        , -4.37583229],
       [ 0.        ,  8.43262129,  0.        , -4.4386235 ]])

In [5]:
mgwr_results.filter_tvals(critical_t=[ 2.51210749,  2.47888792,  2.31069113,  2.21000184])[0:5]

array([[-2.5733093 ,  2.69323024,  0.        , -5.37412032],
       [-3.22998159,  0.        ,  0.        , -5.27052241],
       [-2.85224022,  2.53018155,  0.        , -5.36015537],
       [ 0.        ,  0.        ,  0.        , -4.37583229],
       [ 0.        ,  8.43262129,  0.        , -4.4386235 ]])

In [6]:
mgwr_results.filter_tvals(alpha=[ 0.01300501,  0.01422973,  0.02214297,  0.02854278])[0:5]

array([[-2.5733093 ,  2.69323024,  0.        , -5.37412032],
       [-3.22998159,  0.        ,  0.        , -5.27052241],
       [-2.85224022,  2.53018155,  0.        , -5.36015537],
       [ 0.        ,  0.        ,  0.        , -4.37583229],
       [ 0.        ,  8.43262129,  0.        , -4.4386235 ]])

In [10]:
mgwr_results.resid_deviance

NotImplementedError: deviance not currently used for Gaussian

In [14]:
    gwr.y_fix

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
      

In [26]:
mgwr_results.normalized_cov_params

NotImplementedError: N/A

In [22]:
gwr_results.df_model

147.19523028326989

In [5]:
mgwr_results.spatial_variability(selector,n_iters=5)

[ 0.1711677   0.21630789  0.03589965  0.01871534]
[ 0.1711677   0.21630789  0.03589965  0.01871534]
[ 0.1711677   0.21630789  0.03589965  0.01871534]
[ 0.1711677   0.21630789  0.03589965  0.01871534]
[ 0.1711677   0.21630789  0.03589965  0.01871534]
[ 0.1711677   0.21630789  0.03589965  0.01871534]


array([ 0.,  0.,  0.,  0.])

In [45]:
import copy
seed = 5536
n_iters=5
temp_sel = copy.deepcopy(selector)
temp_mgwr = copy.deepcopy(mgwr)

if seed is None:
    np.random.seed(5536)
else:
    np.random.seed(seed)

fit_params = temp_mgwr.fit_params
search_params = temp_sel.search_params
kernel = temp_mgwr.kernel
fixed = temp_mgwr.fixed


if mgwr.constant:
    X = mgwr.X[:,1:]
else:
    X = mgwr.X

init_sd =  np.std(mgwr_results.params, axis=0)
print(init_sd)
SDs = []

for x in range(n_iters):
    temp_coords = np.random.permutation(mgwr.coords)
    temp_sel.coords = temp_coords
    temp_sel._build_dMat()
    temp_bw = temp_sel.search(**search_params)
    print(temp_bw)
    temp_mgwr.W = temp_mgwr._build_W(fixed, kernel, temp_coords, temp_bw)
    temp_params = temp_sel.est
    print(temp_params[0,:])
    temp_sd = np.std(temp_params, axis=0)
    print(temp_sd)
    SDs.append(temp_sd)

p_vals = (np.sum(np.array(SDs) > init_sd, axis=0) / float(n_iters))


[ 0.1711677   0.21630789  0.03589965  0.01871534]
[131.0, 23.0, 158.0, 158.0]
[-0.12530862  0.0504044  -0.05640277 -0.31512922]
[ 0.0588905   0.28621754  0.00640939  0.01400096]
[158.0, 34.0, 158.0, 67.0]
[-0.0076567   0.15931434 -0.09444184 -0.59458368]
[ 0.00923346  0.23323694  0.00852063  0.11924903]
[109.0, 16.0, 65.0, 122.0]
[-0.09988452  0.0873983  -0.01978968 -0.452024  ]
[ 0.08302842  0.31247016  0.15535661  0.07690364]
[158.0, 154.0, 158.0, 154.0]
[ 0.00374917  0.57576426 -0.07710591 -0.42995375]
[ 0.01124893  0.08790173  0.01561961  0.04924622]
[110.0, 41.0, 85.0, 110.0]
[ 0.10395438  0.19175415 -0.39785243 -0.55697679]
[ 0.06046642  0.22670586  0.12672105  0.08985263]


In [41]:
temp_coords = np.random.permutation(mgwr.coords)
temp_sel.coords = temp_coords
temp_sel._build_dMat()
temp_bw = temp_sel.search(**search_params)
print(temp_bw)
temp_params = temp_gwr.fit(**fit_params).params

[158.0, 21.0, 158.0, 55.0]


In [38]:
temp_mgwr.coords = temp_coords

In [43]:
temp_sel.est

array([[-0.03680033,  0.41830769, -0.04671356, -0.28328242],
       [-0.03331953,  0.30062165, -0.06945034, -0.28856911],
       [-0.02028726,  0.342555  , -0.04560142, -0.12466865],
       [-0.03052891,  0.32398317, -0.05693357, -0.25919062],
       [-0.0280248 ,  0.5984548 , -0.05300768, -0.23057503],
       [-0.02113293,  0.45468774, -0.02878187, -0.328132  ],
       [-0.03154104,  0.30786078, -0.05325849, -0.30066623],
       [-0.02053676,  0.37393471, -0.04166991, -0.15572032],
       [-0.03539109,  0.30351299, -0.04372639, -0.28278051],
       [-0.02868463,  0.32279128, -0.02910886, -0.50384323],
       [-0.03627112,  0.63947648, -0.04772084, -0.31817761],
       [-0.02080522,  1.18637008, -0.03435861, -0.27429531],
       [-0.03681574,  0.54462597, -0.05947354, -0.23126561],
       [-0.02986326,  0.42073669, -0.06666553, -0.31002527],
       [-0.01994273,  0.83679675, -0.02745809, -0.26128986],
       [-0.03102897,  0.2041873 , -0.06225407, -0.27933981],
       [-0.02070083,  0.

In [8]:
mgwr_results.spatial_variability(selector, 5)

array([ 0. ,  0.8,  0.4,  0.8])

In [6]:
mgwr_results.local_collinearity()[0]

array([[ 1.4835582 ],
       [ 1.39416617],
       [ 1.43302695],
       [ 2.51949411],
       [ 2.45378188],
       [ 2.19523169],
       [ 2.27648157],
       [ 2.50756169],
       [ 1.63629209],
       [ 1.44456268],
       [ 2.3816234 ],
       [ 2.36901372],
       [ 1.39638423],
       [ 1.56135598],
       [ 1.5646615 ],
       [ 1.73221934],
       [ 2.13610401],
       [ 2.36491423],
       [ 2.77279923],
       [ 1.41007706],
       [ 1.80126072],
       [ 2.40309843],
       [ 3.05991209],
       [ 1.36150756],
       [ 1.5940654 ],
       [ 2.4269672 ],
       [ 2.76946287],
       [ 2.39132963],
       [ 2.22831611],
       [ 2.89440115],
       [ 2.41091651],
       [ 1.32172493],
       [ 2.32703355],
       [ 1.44081025],
       [ 1.79026045],
       [ 2.32838614],
       [ 1.50754142],
       [ 2.44183404],
       [ 2.27248246],
       [ 2.16130862],
       [ 3.09895529],
       [ 2.390457  ],
       [ 2.35062518],
       [ 2.39379447],
       [ 2.11837379],
       [ 2

In [7]:
gwr_results.local_collinearity()[2]

array([[ 1.5245192 ],
       [ 1.46453283],
       [ 1.48617061],
       [ 2.43696761],
       [ 2.3684091 ],
       [ 2.50071234],
       [ 2.60081364],
       [ 2.42334621],
       [ 1.70294724],
       [ 1.55404727],
       [ 2.27014424],
       [ 2.46594843],
       [ 1.50856223],
       [ 1.67118894],
       [ 1.68399708],
       [ 1.85734003],
       [ 2.31922614],
       [ 2.79403124],
       [ 2.5664562 ],
       [ 1.53332889],
       [ 1.89066743],
       [ 2.80426646],
       [ 2.7633522 ],
       [ 1.46486412],
       [ 1.72670121],
       [ 2.38489605],
       [ 2.55822922],
       [ 2.45583217],
       [ 2.60383749],
       [ 2.62985061],
       [ 2.92521994],
       [ 1.40568829],
       [ 2.6139489 ],
       [ 1.50561013],
       [ 1.87755954],
       [ 2.50346895],
       [ 1.63669664],
       [ 2.94066969],
       [ 2.29004213],
       [ 2.2126658 ],
       [ 2.81634471],
       [ 2.44172316],
       [ 2.2838118 ],
       [ 2.78267762],
       [ 2.18398568],
       [ 2

In [ ]:
selector.